In [4]:
import os, csv, re
from keras.utils import np_utils
from sklearn.preprocessing import normalize
from keras.optimizers import SGD, RMSprop, Adam
import random
SEED = 448
from sklearn import metrics
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Reshape, Lambda, Merge, Embedding
from keras.optimizers import SGD, RMSprop
from keras.regularizers import l2
from keras.layers.core import Dense
from keras.callbacks import EarlyStopping
from keras.layers import Convolution2D, MaxPooling2D, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.wrappers import TimeDistributed
from keras import backend as K
from keras.models import load_model
from keras.layers import LSTM, GRU, SimpleRNN

dimensions 5328  rows: 37 cols= 144

In [26]:
def loadBatch(year):
    print 'loading data: ' + str(year)
    with open('./data_batch/flood_label_' + str(year) + '.csv') as p:
        s = p.read()
    labels = s.split('\n')[1:]
    labels = labels[:-1]
    labels = [float(x.split(',')[1]) for x in labels]
    y_train = np.asarray(labels)
    
    files = os.listdir('./data_batch/')
    # print files
    pw = 8
    t850 = 1
    u300 = 2
    u850 = 3
    v300 = 4
    v850 = 5
    z300 = 6
    z500 = 7
    z1000 = 0

    X = np.zeros((y_train.shape[0], 10, 9, 5328))

    for f in files:
        if f.split('_')[2][:-4] == str(year):
            if f.split('_')[1] == 'z1000':
                with open('./data_batch/' + f) as p:
                    s = p.read()
                    inputs = s.split('\n')[1:]
                    inputs = inputs[:-1]
                    inputs = inputs[:y_train.shape[0]+9]
                    inputs = [x.split(',')[4:] for x in inputs]
                    inputs = np.asarray(inputs)
#                     print inputs.shape
                for r in range(y_train.shape[0]):
                    for i in range(10):
                        X[r][i][z1000] = inputs[r+i]

            if f.split('_')[1] == 't850':
                with open('./data_batch/' + f) as p:
                    s = p.read()
                    inputs = s.split('\n')[1:]
                    inputs = inputs[:-1]
                    inputs = inputs[:y_train.shape[0]+9]
                    inputs = [x.split(',')[4:] for x in inputs]
                    inputs = np.asarray(inputs)
#                     print inputs.shape
                for r in range(y_train.shape[0]):
                    for i in range(10):
                        X[r][i][t850] = inputs[r+i]
            if f.split('_')[1] == 'u300':
                with open('./data_batch/' + f) as p:
                    s = p.read()
                    inputs = s.split('\n')[1:]
                    inputs = inputs[:-1]
                    inputs = inputs[:y_train.shape[0]+9]
                    inputs = [x.split(',')[4:] for x in inputs]
                    inputs = np.asarray(inputs)
#                     print inputs.shape
                for r in range(y_train.shape[0]):
                    for i in range(10):
                        X[r][i][u300] = inputs[r+i]
            if f.split('_')[1] == 'u850':
                with open('./data_batch/' + f) as p:
                    s = p.read()
                    inputs = s.split('\n')[1:]
                    inputs = inputs[:-1]
                    inputs = inputs[:y_train.shape[0]+9]
                    inputs = [x.split(',')[4:] for x in inputs]
                    inputs = np.asarray(inputs)
#                     print inputs.shape
                for r in range(y_train.shape[0]):
                    for i in range(10):
                        X[r][i][u850] = inputs[r+i]
            if f.split('_')[1] == 'v300':
                with open('./data_batch/' + f) as p:
                    s = p.read()
                    inputs = s.split('\n')[1:]
                    inputs = inputs[:-1]
                    inputs = inputs[:y_train.shape[0]+9]
                    inputs = [x.split(',')[4:] for x in inputs]
                    inputs = np.asarray(inputs)
#                     print inputs.shape
                for r in range(y_train.shape[0]):
                    for i in range(10):
                        X[r][i][v300] = inputs[r+i]
            if f.split('_')[1] == 'v850':
                with open('./data_batch/' + f) as p:
                    s = p.read()
                    inputs = s.split('\n')[1:]
                    inputs = inputs[:-1]
                    inputs = inputs[:y_train.shape[0]+9]
                    inputs = [x.split(',')[4:] for x in inputs]
                    inputs = np.asarray(inputs)
#                     print inputs.shape
                for r in range(y_train.shape[0]):
                    for i in range(10):
                        X[r][i][v850] = inputs[r+i]
            if f.split('_')[1] == 'z300':
                with open('./data_batch/' + f) as p:
                    s = p.read()
                    inputs = s.split('\n')[1:]
                    inputs = inputs[:-1]
                    inputs = inputs[:y_train.shape[0]+9]
                    inputs = [x.split(',')[4:] for x in inputs]
                    inputs = np.asarray(inputs)
#                     print inputs.shape
                for r in range(y_train.shape[0]):
                    for i in range(10):
                        X[r][i][z300] = inputs[r+i]
            if f.split('_')[1] == 'z500':
                with open('./data_batch/' + f) as p:
                    s = p.read()
                    inputs = s.split('\n')[1:]
                    inputs = inputs[:-1]
                    inputs = inputs[:y_train.shape[0]+9]
                    inputs = [x.split(',')[4:] for x in inputs]
                    inputs = np.asarray(inputs)
#                     print inputs.shape
                for r in range(y_train.shape[0]):
                    for i in range(10):
                        X[r][i][z500] = inputs[r+i]
            if f.split('_')[1] == 'pw':
                with open('./data_batch/' + f) as p:
                    s = p.read()
                    inputs = s.split('\n')[1:]
                    inputs = inputs[:-1]
                    inputs = inputs[:y_train.shape[0]+9]
                    inputs = [x.split(',')[4:] for x in inputs]
                    inputs = np.asarray(inputs)
#                     print inputs.shape
                for r in range(y_train.shape[0]):
                    for i in range(10):
                        X[r][i][pw] = inputs[r+i]
    X = X.reshape(y_train.shape[0], 10, 9, 37, 144)
    return X, y_train

In [35]:
X_test, y_test = loadBatch(1960)
X_val, y_val = loadBatch(1959)

loading data: 1960
loading data: 1959


In [13]:
print X_test.shape
print y_test.shape
print y_test[:20]

(352, 10, 8, 37, 144)
(352,)
[ 0.444       0.          0.050952    0.27524     0.002381    0.          0.
  0.00090909  0.          0.          0.0018182   0.0018182   0.015909
  0.0013636   0.035       0.          0.          0.          0.          0.        ]


In [ ]:
epoch = 25
nb_filter1 = 16
nb_filter2 = 32
hidden = 16
min_loss = 1000000
model = Sequential()
model.add(TimeDistributed(Convolution2D(nb_filter1, 5, 5, border_mode='valid', subsample=(1,3)), 
                               input_shape=(10, 9, 37, 144)))
model.add(Activation('relu'))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2),border_mode='valid')))
model.add(TimeDistributed(Convolution2D(nb_filter2, 3, 3, border_mode='valid', subsample=(2,3))))
model.add(Activation('relu'))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2),border_mode='valid')))
model.add(TimeDistributed(Flatten()))
model.add(SimpleRNN(output_dim=hidden))
model.add(Dropout(.5))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('relu'))
model.compile(loss='mse',
              optimizer=RMSprop(lr=0.0002, rho=0.9, epsilon=1e-08),
              metrics=['accuracy'])
for e in range(epoch):
    print '===========' + 'epoch ' + str(e) + '==========='
    for i in range(10):
        X_train, y_train = loadBatch(1948+i)
        print 'finish loading'
        hist = model.fit(X_train,
                  y_train,
                  nb_epoch=1, 
                  validation_data = (X_val, y_val), 
                  shuffle=True)
        cur_loss = hist.history['val_loss'][0]
        if cur_loss < min_loss:
            min_loss = cur_loss
            model.save('model/cnn_lstm.h5')
            
model = load_model('model/cnn_lstm.h5')
score = model.evaluate(X_test, y_test)
print score

===========epoch 0===========
loading data: 1948
finish loading
Train on 352 samples, validate on 351 samples
Epoch 1/1
352/352 [==============================] - 3s - loss: 0.0427 - acc: 0.4062 - val_loss: 0.0527 - val_acc: 0.2821
loading data: 1949


In [34]:
model = load_model('model/cnn_lstm.h5')
score = model.evaluate(X_test, y_test)
print score
# model.summary()

352/352 [==============================] - 1s     
[0.036504719787361944, 0.36079545454545453]


validation split in each batch [0.036504719787361944, 0.36079545454545453]
LSTM fix validation set 1959 [0.031095882793041794, 0.36079545454545453]

In [29]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
timedistributed_26 (TimeDistribut(None, 10, 32, 33, 47)7232        timedistributed_input_9[0][0]    
____________________________________________________________________________________________________
activation_23 (Activation)       (None, 10, 32, 33, 47)0           timedistributed_26[0][0]         
____________________________________________________________________________________________________
timedistributed_27 (TimeDistribut(None, 10, 32, 16, 23)0           activation_23[0][0]              
____________________________________________________________________________________________________
timedistributed_28 (TimeDistribut(None, 10, 16, 14, 7) 4624        timedistributed_27[0][0]         
___________________________________________________________________________________________